In [2]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re

from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback,
    BartConfig,
    BartForConditionalGeneration

)
from datasets import Dataset
from tqdm import tqdm


In [4]:
chckpoint = "alaggung/bart-pretrained"
config = BartConfig.from_pretrained(chckpoint)
model = BartForConditionalGeneration(config)
#model = AutoModelForSeq2SeqLM.from_pretrained(chckpoint) 
tokenizer = AutoTokenizer.from_pretrained(chckpoint)

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (..) 제거
    #sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    # 스폐셜 토큰 적용할 거면 여기 위에 영어 외 문자 공백으로 만들때 스폐셜 토큰을 넘어갈 수 있도록 지정해주면된다.
    # 그리고 세번째 정규표현식을 지워야 ||할 것이다. 
    return sentence


In [6]:
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [7]:
train_df.head()

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)


In [8]:
train_text = []
val_text = []

for tt in tqdm(train_df['Text']):
    train_text.append(preprocess_sentence(tt))

for vt in tqdm(val_df['Text']):
    val_text.append(preprocess_sentence(vt))

100%|█████████████████████████████████████████████████████████████████| 35004/35004 [00:01<00:00, 31481.95it/s]


In [9]:
train_Category = []
val_Category= []

for tc in tqdm(train_df['Category']):
    train_Category.append(tc)

for vc in tqdm(val_df['Category']):
    val_Category.append(vc)

100%|███████████████████████████████████████████████████████████████| 35004/35004 [00:00<00:00, 2451205.71it/s]


In [10]:
for i in range(len(val_text)):
    temp_cat = "#"+val_Category[i]+"# "
    val_text[i] = temp_cat+val_text[i]
    
for i in range(len(train_text)):
    temp_cat = "#"+train_Category[i]+"# "
    train_text[i] = temp_cat+train_text[i]

In [11]:
train_df = pd.DataFrame(zip(train_text), columns=['Text'])
val_df = pd.DataFrame(zip(val_text), columns=['Text'])

In [12]:
train_df.head()

,Text
0,#상거래(쇼핑)# 그럼 날짜는 가격 큰 변동 없으면 6 28 7 13로 확정할까 우...
1,#상거래(쇼핑)# kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마...
2,#상거래(쇼핑)# 아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 ...
3,#상거래(쇼핑)# 칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남 아 그 왕...
4,#상거래(쇼핑)# 잠도안오네 얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보...


In [13]:
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)

In [14]:
print(train_data)
print(val_data)

Dataset(features: {'Text': Value(dtype='string', id=None)}, num_rows: 279992)
Dataset(features: {'Text': Value(dtype='string', id=None)}, num_rows: 35004)


In [14]:
max_input = 128
max_target = 128
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [15]:
def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [16]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [17]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Text'])):
        label_id.append(tokenizer.encode(data_to_process['Text'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Text'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Text'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [18]:
special_words = [
                "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
                "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
                "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
                "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
                "#개인 및 관계#", "#미용과 건강#", "#상거래(쇼핑)#", "#시사/교육#", "#식음료#", 
                "#여가 생활#", "#일과 직업#", "#주거와 생활#", "#행사#",
                ]

tokenizer.add_special_tokens({"additional_special_tokens": special_words})
model.resize_token_embeddings(len(tokenizer))


model.config.max_length = 128 
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5

In [19]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text'])


  0%|          | 0/280 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

In [20]:
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     print("labels_ids",labels_ids)
#     print("labels_ids[labels_ids == -100]",labels_ids[labels_ids == -100])
#     print("tokenizer.pad_token_id",tokenizer.pad_token_id)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].high
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].high
    rouge_outputL = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].high
    

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
        
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4), 
        
        "rougeL_precision": round(rouge_outputL.precision, 4),
        "rougeL_recall": round(rouge_outputL.recall, 4),
        "rougeL_fmeasure": round(rouge_outputL.fmeasure, 4),
    }

/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/non_MLM_totaldata_test",
    num_train_epochs=50, # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-5,
    weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch'


    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [22]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [23]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
trainer.train()

/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 279992
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 875000
  Number of trainable parameters = 46719488
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/checkpoint-17500
Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-17500/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-17500/pytorch_model.bin
tokenizer config file saved in checkpoint/non_MLM_totaldata_test/checkpoint-17500/tokenizer_config.json
Special tokens file saved in checkpoint/non_MLM_totaldata_test/checkpoint-17500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/checkpoint-35000
Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-35000/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-35000/pytorch_model.bin
tokenizer config file saved in checkpoint/non_MLM_totaldata_test/checkpoint-35000/tokenizer_config.json
Special tokens file saved in che

Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-245000/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-245000/pytorch_model.bin
tokenizer config file saved in checkpoint/non_MLM_totaldata_test/checkpoint-245000/tokenizer_config.json
Special tokens file saved in checkpoint/non_MLM_totaldata_test/checkpoint-245000/special_tokens_map.json
Deleting older checkpoint [checkpoint/non_MLM_totaldata_test/checkpoint-192500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/checkpoint-262500
Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-262500/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-262500/pytorch_model.bin
tokenizer config file saved in checkpoint/non_MLM_totaldata_test/checkpoint-262500/tokenizer_config.json
Special tokens file saved in checkpoint/non_MLM_totaldata_test/ch

***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/checkpoint-472500
Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-472500/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-472500/pytorch_model.bin
tokenizer config file saved in checkpoint/non_MLM_totaldata_test/checkpoint-472500/tokenizer_config.json
Special tokens file saved in checkpoint/non_MLM_totaldata_test/checkpoint-472500/special_tokens_map.json
Deleting older checkpoint [checkpoint/non_MLM_totaldata_test/checkpoint-420000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/checkpoint-490000
Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-490000/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-490000/pytorch_model.bin
tokenizer config fi

Special tokens file saved in checkpoint/non_MLM_totaldata_test/checkpoint-682500/special_tokens_map.json
Deleting older checkpoint [checkpoint/non_MLM_totaldata_test/checkpoint-630000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/checkpoint-700000
Configuration saved in checkpoint/non_MLM_totaldata_test/checkpoint-700000/config.json
Model weights saved in checkpoint/non_MLM_totaldata_test/checkpoint-700000/pytorch_model.bin
tokenizer config file saved in checkpoint/non_MLM_totaldata_test/checkpoint-700000/tokenizer_config.json
Special tokens file saved in checkpoint/non_MLM_totaldata_test/checkpoint-700000/special_tokens_map.json
Deleting older checkpoint [checkpoint/non_MLM_totaldata_test/checkpoint-647500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/non_MLM_totaldata_test/che

TrainOutput(global_step=875000, training_loss=2.635788450892857, metrics={'train_runtime': 100047.073, 'train_samples_per_second': 139.93, 'train_steps_per_second': 8.746, 'total_flos': 4.745855691128832e+17, 'train_loss': 2.635788450892857, 'epoch': 50.0})